In [73]:
import json
import requests
import pandas as pd
from datetime import date, timedelta
import dateutil.relativedelta
import os
auth={'Authorization': "64c57c627bfd4083b6bfa8a65272e3a3"}
columns=['month','page_name','page_id','component_name','uptime','major_outages','partial_outages']
uptime_final=pd.DataFrame(columns=['month','page_name','page_id','component_name','uptime','major_outages','partial_outages'])
incident_final=pd.DataFrame(columns=['uptime_id','incident_id','page_id'])
pages=requests.get(url='https://api.statuspage.io/v1/pages',headers=auth)
component_url="https://api.statuspage.io/v1/pages/{page_id}/components"
uptime_url="https://api.statuspage.io/v1/pages/{page_id}/components/{component_id}/uptime"
incident_url="https://api.statuspage.io/v1/pages/{page_id}/incidents/{incident_id}"
first_day_of_current_month = date.today().replace(day=1)
last_day_of_previous_month = first_day_of_current_month - timedelta(days=1)
first_day_of_previous_month= first_day_of_current_month+ dateutil.relativedelta.relativedelta(months=-1)
month=first_day_of_previous_month.strftime('%B')+"-"+first_day_of_previous_month.strftime('%Y')
print(month)
first_day_of_previous_month=first_day_of_previous_month.strftime("%Y-%m-%d")
last_day_of_previous_month=last_day_of_previous_month.strftime("%Y-%m-%d")



params={'start': "2024-02-01",
        'end': "2024-02-29"}
for page in pages.json():
    page_name=page["name"]
    #print(page_name)
    page_id=page["id"]
    url_component=component_url.format(page_id=page_id)
    #url_incident=incident_url.format(page_id=page_id)
    if(page_name  in ["webMethods.io iPaaS"]):
        # incidents=requests.get(url=url_incident,headers=auth)
        # incidents=incidents.json()
        # for incident in incidents:
        #     print(incident["name"],"########",incident["impact"])
        #     print("_______________________")
        params_component={'per_page':100 }
        for page in range(1,5):
            params_component.update({"page":page})
            components=requests.get(url=url_component,headers=auth,params=params_component)
            print(components.json())
            for component in components.json():
                component_id=component["id"]
                component_name=component["name"]
                url_uptime=uptime_url.format(page_id=page_id,component_id=component_id)
                uptime_json=requests.get(url=url_uptime,headers=auth,params=params)
                uptime_json=uptime_json.json()
    #             print(uptime_json)
    #             print("#####################")
                try:
                    uptime=uptime_json["uptime_percentage"]
                    major_outages=uptime_json["major_outage"]
                    partial_outage=uptime_json["partial_outage"]
                    append_dict_uptime={'id': uptime_json["id"] ,'month': month,'page_name': page_name,'page_id': page_id,'component_name': component_name,
                    'uptime': uptime,'major_outages': major_outages,'partial_outages': partial_outage}
                    append_dict_incident={}
                    for incident in uptime_json["related_events"]:
                        append_dict_incident.update({"uptime_id": uptime_json["id"], "incident_id": incident["id"],"page_id": page_id  })
                    append_df_uptime= pd.DataFrame([append_dict_uptime])
                    uptime_final=pd.concat([uptime_final,append_df_uptime])
                    append_df_incident=pd.DataFrame([append_dict_incident])
                    incident_final=pd.concat([incident_final,append_df_incident])
                except Exception as e:
                    print("exception",e,component_name)
                    #print(uptime_json)
                    print("########################")
                    pass

February-2024
[{'id': '6jzj397l1ggq', 'page_id': 'qr0xdjynnv32', 'group_id': '2bympz7x99hp', 'created_at': '2021-02-18T15:17:20Z', 'updated_at': '2024-02-20T09:00:56Z', 'group': False, 'name': 'US1 WMIO INT', 'description': None, 'position': 1, 'status': 'operational', 'showcase': True, 'only_show_if_degraded': False, 'automation_email': 'component+26c9c974-3622-430e-8543-60b9a5624967@notifications.statuspage.io', 'start_date': '2021-02-17'}, {'id': 'w4148jxhtq6t', 'page_id': 'qr0xdjynnv32', 'group_id': None, 'created_at': '2021-02-19T10:39:40Z', 'updated_at': '2022-12-01T09:59:20Z', 'group': True, 'name': 'Software AG Cloud', 'description': None, 'position': 1, 'status': 'operational', 'showcase': False, 'only_show_if_degraded': False, 'automation_email': 'component+edacada3-d794-477e-a8ff-01a7b9bae409@notifications.statuspage.io', 'start_date': None}, {'id': '2lct1sdn15wz', 'page_id': 'qr0xdjynnv32', 'group_id': 'w4148jxhtq6t', 'created_at': '2021-02-19T10:47:15Z', 'updated_at': '202

/var/folders/ff/hqsbywlj7pqfrwcx68lbt1680000gp/T/ipykernel_11741/2436144571.py:62: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  uptime_final=pd.concat([uptime_final,append_df_uptime])


exception 'uptime_percentage' Software AG Cloud
########################
exception 'uptime_percentage' webMethods.io Integration
########################
exception 'uptime_percentage' webMethods.io API Gateway
########################
exception 'uptime_percentage' webMethods.io Developer Portal
########################
exception 'uptime_percentage' webMethods.io B2B
########################
exception 'uptime_percentage' webMethods.io MFT
########################
exception 'uptime_percentage' webMethods.io End-to-End Monitoring
########################
exception 'uptime_percentage' webMethods.io Embed
########################
exception 'uptime_percentage' webMethods Cloud Container
########################
exception 'uptime_percentage' webMethods Agile Apps
########################
exception 'uptime_percentage' webMethods.io API Control Plane
########################
exception 'uptime_percentage' webMethods.io AI
########################
[]
[]
[]


In [3]:
print(uptime_final)
print(incident_final)
incidents=incident_final.incident_id.unique()
print(incidents)

            month                           page_name       page_id  \
0   February-2024                 webMethods.io iPaaS  qr0xdjynnv32   
0   February-2024                 webMethods.io iPaaS  qr0xdjynnv32   
0   February-2024                 webMethods.io iPaaS  qr0xdjynnv32   
0   February-2024                 webMethods.io iPaaS  qr0xdjynnv32   
0   February-2024                 webMethods.io iPaaS  qr0xdjynnv32   
..            ...                                 ...           ...   
0   February-2024  webMethods.io iPaaS - customers 01  smcmk2ztmnb3   
0   February-2024  webMethods.io iPaaS - customers 01  smcmk2ztmnb3   
0   February-2024  webMethods.io iPaaS - customers 01  smcmk2ztmnb3   
0   February-2024  webMethods.io iPaaS - customers 01  smcmk2ztmnb3   
0   February-2024  webMethods.io iPaaS - customers 01  smcmk2ztmnb3   

                             component_name  uptime major_outages  \
0                              US1 WMIO INT   100.0             0   
0        

In [56]:
from datetime import datetime
incident_detail=pd.DataFrame(columns=['incident_id','page_id','incident','impact','start_time','end_time','duration'])
for incident in incidents:
    incident_temp=incident_final.loc[incident_final["incident_id"] == incident]
    if(incident_temp.shape[0]>0):
        page_id=incident_temp.iloc[0]["page_id"]
        url_incident=incident_url.format(page_id=page_id,incident_id=incident)
        incident_json=requests.get(url=url_incident,headers=auth)
        incident_json=incident_json.json()
        start_time=""
        for update in incident_json["incident_updates"]:
            if(update["status"] == "resolved" or update["status"] == "completed" ):
                end_time=datetime.strptime(update["created_at"],"%Y-%m-%dT%H:%M:%S.%fZ")
            if(incident_json["impact"]=="maintenance" and update["status"] == "in_progress"):
                start_time=datetime.strptime(update["created_at"],"%Y-%m-%dT%H:%M:%S.%fZ")
            if((incident_json["impact"]=="major" or incident_json["impact"]=="critical") and update["status"] == "investigating" ):
                start_time=datetime.strptime(update["created_at"],"%Y-%m-%dT%H:%M:%S.%fZ")

        if not start_time:
            start_time=datetime.strptime(incident_json["created_at"],"%Y-%m-%dT%H:%M:%SZ")
        td =end_time-start_time
        print(td)
        print(start_time,end_time,incident_json["impact"])
        append_dict_inc={'incident_id': incident,'page_id': page_id,'incident': incident_json["name"],'impact': incident_json["impact"]
                ,'start_time': start_time,'end_time': end_time,'duration': td.total_seconds() }
        append_df_inc= pd.DataFrame([append_dict_inc])
        incident_detail=pd.concat([incident_detail,append_df_inc])
        
        
    


2:35:03.698000
2024-02-17 16:01:53.567000 2024-02-17 18:36:57.265000 major


/var/folders/ff/hqsbywlj7pqfrwcx68lbt1680000gp/T/ipykernel_11741/1641275248.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  incident_detail=pd.concat([incident_detail,append_df_inc])


6 days, 19:07:16.142000
2024-01-25 10:16:55 2024-02-01 05:24:11.142000 maintenance
4:00:00.093000
2024-02-26 02:00:56.392000 2024-02-26 06:00:56.485000 maintenance
2:44:37.220000
2024-02-17 00:00:56.635000 2024-02-17 02:45:33.855000 maintenance
3:00:00.060000
2024-02-25 07:30:56.158000 2024-02-25 10:30:56.218000 maintenance
1:09:46.634000
2024-02-27 02:58:33.498000 2024-02-27 04:08:20.132000 critical
0:29:59.590000
2024-02-21 08:00:57.102000 2024-02-21 08:30:56.692000 maintenance
6:00:02.094000
2024-02-17 03:00:56.886000 2024-02-17 09:00:58.980000 maintenance
0:30:00.191000
2024-02-26 05:00:56.283000 2024-02-26 05:30:56.474000 maintenance
0:29:59.755000
2024-02-22 08:00:56.561000 2024-02-22 08:30:56.316000 maintenance
6 days, 19:07:17.598000
2024-01-25 10:16:55 2024-02-01 05:24:12.598000 maintenance
2:30:19.452000
2024-02-24 00:30:56.577000 2024-02-24 03:01:16.029000 maintenance
1:00:00.059000
2024-02-21 03:00:57.393000 2024-02-21 04:00:57.452000 maintenance
1:42:25.322000
2024-02-15 0

In [57]:
print(incident_detail)

    incident_id       page_id  \
0  21thd3vm0gg8  qr0xdjynnv32   
0  shss9t19nd65  qr0xdjynnv32   
0  dhgy3c9zz89x  qr0xdjynnv32   
0  xk6l5xf4jllk  qr0xdjynnv32   
0  1mcs6hbpfn0l  qr0xdjynnv32   
0  ggxp6n18qrd6  qr0xdjynnv32   
0  jlxdlprcr0t7  qr0xdjynnv32   
0  6r9zfkd99q3z  qr0xdjynnv32   
0  h2wtvnywkn8c  qr0xdjynnv32   
0  9frpvj5k96mv  qr0xdjynnv32   
0  6x24r70xnckj  qr0xdjynnv32   
0  zr7pxkc807nx  qr0xdjynnv32   
0  rhbm9sw2f4ys  qr0xdjynnv32   
0  rf4261bdkkjk  qr0xdjynnv32   
0  60c8nglm46fb  qr0xdjynnv32   
0  6dhhcpqx95y7  qr0xdjynnv32   
0  sxmyzl43dh2m  qr0xdjynnv32   
0  tbg5t95kv7z6  qr0xdjynnv32   
0  030n4sk4nnnc  qr0xdjynnv32   
0  thgy7q3tqkrv  qr0xdjynnv32   
0  x6407sdznwr7  qr0xdjynnv32   
0  d429tllvkpdt  qr0xdjynnv32   
0  y907nvjs2pn6  qr0xdjynnv32   
0  3k35nkn038dx  smcmk2ztmnb3   
0  v5kf9ntqldtb  smcmk2ztmnb3   
0  kh2mk6p5hrvc  smcmk2ztmnb3   
0  mkls7f4y71bw  smcmk2ztmnb3   
0  r15pjxft3q6q  smcmk2ztmnb3   
0  78v08wd8dz09  smcmk2ztmnb3   
0  qfpmbbb